# Compare $D_{ij}$ sums

Calculate for Table 1 in the paper.

In [1]:
%matplotlib notebook

In [2]:
import os
import sys
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter, AutoMinorLocator)
import pandas as pd

In [3]:
os.chdir('..')
os.getcwd()

'C:\\Users\\pfschus\\Box Sync\\Projects\\fnpc\\analysis'

In [4]:
sys.path.append('../scripts/')

In [5]:
import bicorr as bicorr
import bicorr_e as bicorr_e
import bicorr_plot as bicorr_plot
import bicorr_math as bicorr_math
import bicorr_sums as bicorr_sums

# Load `det_df`

In [6]:
det_df = bicorr.load_det_df('../meas_info/det_df_pairs_angles.csv')
det_df.head()

,d1,d2,d1d2,angle
0,1,2,102,15.0
1,1,3,103,30.0
2,1,4,104,45.0
3,1,5,105,60.0
4,1,6,106,75.0


# Set energy range

In [7]:
e_min = 1
e_max = 4

# Load `bhm_e` data

In [8]:
legends =['Experiment', 'CGMF',      'FREYA',      'PoliMi',    'PoliMi-No CT']
fmts   = ['x',          's',         'D',          'o',         '^']
colors = ['#5d269b',    '#dd673b',   '#80bc31',    '#3cbfe0',   '#4242f4']

to_plot = [0,1,2,3,4]

In [9]:
bhm_e_meas, e_bin_edges, note = bicorr_e.load_bhm_e('../analysis/Cf072115_to_Cf072215b/datap'); print(note)
bhm_e_cgmf, e_bin_edges, note = bicorr_e.load_bhm_e('../analysis/cgmf/datap'); print(note)
bhm_e_freya,e_bin_edges, note = bicorr_e.load_bhm_e('../analysis/freya/datap'); print(note)
bhm_e_ipol, e_bin_edges, note = bicorr_e.load_bhm_e('../analysis/ipol/datap'); print(note)
bhm_e_ipol_noct, e_bin_edges, note = bicorr_e.load_bhm_e('../analysis/ipol_noct/datap'); print(note)

Combined across folders
Original file: bicorrCGMF
Original file: bicorrFREYAaf
Original file: bicorrIPOLac
Original file: bicorrIPOLac


In [10]:
index = bicorr.generate_pair_is(det_df, ignore_fc_neighbors_flag=False)
print(index.shape)
bhp_e_meas = bicorr_e.build_bhp_e(bhm_e_meas, e_bin_edges, pair_is = index)[0]
bhp_e_cgmf = bicorr_e.build_bhp_e(bhm_e_cgmf, e_bin_edges, pair_is = index)[0]
bhp_e_freya= bicorr_e.build_bhp_e(bhm_e_freya,e_bin_edges, pair_is = index)[0]
bhp_e_ipol = bicorr_e.build_bhp_e(bhm_e_ipol, e_bin_edges, pair_is = index)[0]
bhp_e_ipol_noct = bicorr_e.build_bhp_e(bhm_e_ipol_noct, e_bin_edges, pair_is = index)[0]
bhp_es = [bhp_e_meas,
          bhp_e_cgmf, 
          bhp_e_freya, 
          bhp_e_ipol, 
          bhp_e_ipol_noct]

(990,)


Load `num_fission`

In [11]:
num_fission_meas = int(int(sio.loadmat('Cf072115_to_Cf072215b/datap/num_fissions.mat')['num_fissions'])*float(sio.loadmat('Cf072115_to_Cf072215b/datap/fc_efficiency.mat')['fc_efficiency']))
num_fission_cgmf = int(sio.loadmat('cgmf/datap/num_fissions.mat')['num_fissions'])
num_fission_freya= int(sio.loadmat('freya/datap/num_fissions.mat')['num_fissions'])
num_fission_ipol = int(sio.loadmat('ipol/datap/num_fissions.mat')['num_fissions'])
num_fission_ipol_noct = int(sio.loadmat('ipol_noct/datap/num_fissions.mat')['num_fissions'])
num_fissions = [num_fission_meas,
                num_fission_cgmf,
                num_fission_freya,
                num_fission_ipol,
                num_fission_ipol_noct]
print(num_fissions)

[1422133977, 192000000, 1000000000, 1000000000, 1000000000]


# Set up a table for storing data

In [12]:
counts_df = pd.DataFrame({'dataset':legends,'num_fissions':num_fissions},index=to_plot)
counts_df['Cd'] = np.nan
counts_df['Cd_err'] = np.nan
counts_df.head()

,dataset,num_fissions,Cd,Cd_err
0,Experiment,1422133977,NaN,NaN
1,CGMF,192000000,NaN,NaN
2,FREYA,1000000000,NaN,NaN
3,PoliMi,1000000000,NaN,NaN
4,PoliMi-No CT,1000000000,NaN,NaN


In [13]:
for i in to_plot:
    bhp_e = bhp_es[i]
    counts_df.loc[i,'Cd'], counts_df.loc[i,'Cd_err'], energies_real = bicorr_sums.calc_nn_sum_e(bhp_e, e_bin_edges, e_min = e_min, e_max = e_max, return_real_energies_flag=True)
counts_df

,dataset,num_fissions,Cd,Cd_err
0,Experiment,1422133977,3941986.0,1985.443527
1,CGMF,192000000,726824.0,852.539735
2,FREYA,1000000000,2978063.0,1725.706522
3,PoliMi,1000000000,3409458.0,1846.471771
4,PoliMi-No CT,1000000000,3276436.0,1810.092815


In [15]:
counts_df['Cd per million fission'] = 10**6 * counts_df['Cd'] / counts_df['num_fissions']
counts_df['Cd_err per million fission'] = 10**6 * counts_df['Cd_err'] / counts_df['num_fissions']

In [16]:
counts_df

,dataset,num_fissions,Cd,Cd_err,Cd per million fission,Cd_err per million fission
0,Experiment,1422133977,3941986.0,1985.443527,2771.880894,1.396102
1,CGMF,192000000,726824.0,852.539735,3785.541667,4.440311
2,FREYA,1000000000,2978063.0,1725.706522,2978.063000,1.725707
3,PoliMi,1000000000,3409458.0,1846.471771,3409.458000,1.846472
4,PoliMi-No CT,1000000000,3276436.0,1810.092815,3276.436000,1.810093


In [17]:
counts_df.to_csv(r'compare/doubles_counts_df.csv')

In [20]:
print(counts_df.to_latex(columns=['dataset','num_fissions','Cd','Cd_err','Cd per million fission','Cd_err per million fission'],index=False))

\begin{tabular}{lrrrrr}
\toprule
      dataset &  num\_fissions &         Cd &       Cd\_err &  Cd per million fission &  Cd\_err per million fission \\
\midrule
   Experiment &    1422133977 &  3941986.0 &  1985.443527 &             2771.880894 &                    1.396102 \\
         CGMF &     192000000 &   726824.0 &   852.539735 &             3785.541667 &                    4.440311 \\
        FREYA &    1000000000 &  2978063.0 &  1725.706522 &             2978.063000 &                    1.725707 \\
       PoliMi &    1000000000 &  3409458.0 &  1846.471771 &             3409.458000 &                    1.846472 \\
 PoliMi-No CT &    1000000000 &  3276436.0 &  1810.092815 &             3276.436000 &                    1.810093 \\
\bottomrule
\end{tabular}

